### loading JSON files into Influx DB- checking every 5 minutes and uploading into database

In [2]:
import os
import time
import json
import shutil
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

# InfluxDB credentials
influxdb_url = "http://localhost:8086"
influxdb_token = "wuJFPquNOH38O6cLOSQNLGPa1SiJ65XH0xPryNNDV9MaHlO9b5FUQOV6fLwDH_RmfNL0GMk7U4srHSZFhUoXDw=="  
influxdb_org = "sjsu"
influxdb_bucket = "ArbitrageBucket"
timeout=30  # Increase the timeout to 30 seconds


# Folder containing JSON files
folder_path = "/Users/priyasuresh/Documents/Documents_Priya’s MacBook Pro/SPRING2024/Data 228-Big Data/project/last_folder/clusters.json"
old_json_path = "/Users/priyasuresh/Documents/Documents_Priya’s MacBook Pro/SPRING2024/Data 228-Big Data/project/old_json"


# Ensure the old_json folder exists
os.makedirs(old_json_path, exist_ok=True)

# Function to load JSON files to InfluxDB
def load_json_files_to_influxdb(client):
    write_api = client.write_api(write_options=SYNCHRONOUS)
    files = [f for f in os.listdir(folder_path) if f.endswith(".json") and not f.startswith("_")]
    for filename in files:
        full_path = os.path.join(folder_path, filename)
        temp_path = os.path.join(old_json_path, filename)
        shutil.move(full_path, temp_path)
        try:
            points = []
            with open(temp_path, 'r') as file:
                for line in file:
                    record = json.loads(line)
                    point = Point("crypto_trade")\
                        .tag("symbol_id", record["symbol_id"])\
                        .tag("taker_side", record["taker_side"])\
                        .field("uuid", record["uuid"])\
                        .field("price", record["price"])\
                        .field("min_buy_price", record["min_buy_price"])\
                        .field("max_sell_price", record["max_sell_price"])\
                        .field("price_diff", record["price_diff"])\
                        .field("time_cluster", record["time_cluster"])\
                        .time(record["time_exchange"], WritePrecision.MS)
                    points.append(point)
            write_api.write(bucket=influxdb_bucket, org=influxdb_org, record=points)
            print(f"Data from {filename} has been written to InfluxDB.")
        except Exception as e:
            print(f"An error occurred with file {filename}: {e}")
            shutil.move(temp_path, full_path)  # Move the file back if there was an error

# Main loop to continuously check the folder every 5 minutes
def main():
    with InfluxDBClient(url=influxdb_url, token=influxdb_token, org=influxdb_org) as client:
        try:
            while True:
                load_json_files_to_influxdb(client)
                time.sleep(300)  # Sleep for 5 minutes
        except KeyboardInterrupt:
            print("Data loading interrupted by user.")
        finally:
            client.close()
            print("InfluxDB client closed.")

if __name__ == "__main__":
    main()


Data from part-00003-f48de372-ea7e-4600-8e4d-cc636ca82844-c000.json has been written to InfluxDB.
Data from part-00006-f48de372-ea7e-4600-8e4d-cc636ca82844-c000.json has been written to InfluxDB.
Data from part-00004-f48de372-ea7e-4600-8e4d-cc636ca82844-c000.json has been written to InfluxDB.
Data from part-00001-f48de372-ea7e-4600-8e4d-cc636ca82844-c000.json has been written to InfluxDB.
Data from part-00000-f48de372-ea7e-4600-8e4d-cc636ca82844-c000.json has been written to InfluxDB.
Data from part-00008-f48de372-ea7e-4600-8e4d-cc636ca82844-c000.json has been written to InfluxDB.
Data from part-00005-f48de372-ea7e-4600-8e4d-cc636ca82844-c000.json has been written to InfluxDB.
Data from part-00007-f48de372-ea7e-4600-8e4d-cc636ca82844-c000.json has been written to InfluxDB.
Data from part-00002-f48de372-ea7e-4600-8e4d-cc636ca82844-c000.json has been written to InfluxDB.
Data from part-00012-b39cde6a-0d0a-4bca-bca8-f2fda762e996-c000.json has been written to InfluxDB.
Data from part-00081

### Final Code to run 

In [1]:
import os
import time
import json
import shutil
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

# InfluxDB credentials
influxdb_url = "http://localhost:8086"
influxdb_token = "wuJFPquNOH38O6cLOSQNLGPa1SiJ65XH0xPryNNDV9MaHlO9b5FUQOV6fLwDH_RmfNL0GMk7U4srHSZFhUoXDw=="  
influxdb_org = "sjsu"
influxdb_bucket = "ArbitrageBucket"

# Folder containing JSON files
folder_path = "/Users/priyasuresh/Documents/Documents_Priya’s MacBook Pro/SPRING2024/Data 228-Big Data/project/last_folder/clusters.json"
old_json_path = "/Users/priyasuresh/Documents/Documents_Priya’s MacBook Pro/SPRING2024/Data 228-Big Data/project/old_json"

# Ensure the old_json folder exists
os.makedirs(old_json_path, exist_ok=True)

# Function to load JSON files to InfluxDB
def load_json_files_to_influxdb(client):
    write_api = client.write_api(write_options=SYNCHRONOUS)
    files = [f for f in os.listdir(folder_path) if f.endswith(".json") and not f.startswith("_")]
    for filename in files:
        full_path = os.path.join(folder_path, filename)
        temp_path = os.path.join(old_json_path, filename)
        shutil.move(full_path, temp_path)
        try:
            with open(temp_path, 'r') as file:
                points = [Point("crypto_trade")
                          .tag("symbol_id", record["symbol_id"])
                          .tag("taker_side", record["taker_side"])
                          .tag("uuid", record["uuid"])  # Using UUID as a tag to ensure uniqueness
                          .field("price", record["price"])
                          .field("min_buy_price", record["min_buy_price"])
                          .field("max_sell_price", record["max_sell_price"])
                          .field("price_diff", record["price_diff"])
                          .field("time_cluster", record["time_cluster"])
                          .time(record["time_exchange"], WritePrecision.MS)
                          for record in (json.loads(line) for line in file)]
            write_api.write(bucket=influxdb_bucket, record=points)
            print(f"Data from {filename} has been written to InfluxDB.")
        except Exception as e:
            print(f"An error occurred with file {filename}: {e}")
            # Move the file back if there was an error
            shutil.move(temp_path, full_path)

# Main loop to continuously check the folder every 5 minutes
def main():
    with InfluxDBClient(url=influxdb_url, token=influxdb_token, org=influxdb_org) as client:
        try:
            while True:
                load_json_files_to_influxdb(client)
                time.sleep(300)  # Sleep for 5 minutes
        except KeyboardInterrupt:
            print("Data loading interrupted by user.")
        finally:
            print("InfluxDB client closed.")

if __name__ == "__main__":
    main()


Data from part-00068-c69c1285-0ed8-43ba-8f24-ee369aa5435d-c000.json has been written to InfluxDB.
Data from part-00065-c69c1285-0ed8-43ba-8f24-ee369aa5435d-c000.json has been written to InfluxDB.
Data from part-00030-c69c1285-0ed8-43ba-8f24-ee369aa5435d-c000.json has been written to InfluxDB.
Data from part-00025-c69c1285-0ed8-43ba-8f24-ee369aa5435d-c000.json has been written to InfluxDB.
Data from part-00028-c69c1285-0ed8-43ba-8f24-ee369aa5435d-c000.json has been written to InfluxDB.
Data from part-00070-c69c1285-0ed8-43ba-8f24-ee369aa5435d-c000.json has been written to InfluxDB.
Data from part-00002-c69c1285-0ed8-43ba-8f24-ee369aa5435d-c000.json has been written to InfluxDB.
Data from part-00057-c69c1285-0ed8-43ba-8f24-ee369aa5435d-c000.json has been written to InfluxDB.
Data from part-00101-c69c1285-0ed8-43ba-8f24-ee369aa5435d-c000.json has been written to InfluxDB.
Data from part-00091-c69c1285-0ed8-43ba-8f24-ee369aa5435d-c000.json has been written to InfluxDB.
Data from part-00042

In [14]:
import os
import time
import json
import shutil
from influxdb_client import InfluxDBClient, Point, WritePrecision
# Only import SYNCHRONOUS if it's available in your influxdb_client package
# from influxdb_client.client.write_api import SYNCHRONOUS

# Your InfluxDB credentials
influxdb_url = "http://localhost:8086"
influxdb_token = "wuJFPquNOH38O6cLOSQNLGPa1SiJ65XH0xPryNNDV9MaHlO9b5FUQOV6fLwDH_RmfNL0GMk7U4srHSZFhUoXDw=="
influxdb_org = "sjsu"
influxdb_bucket = "ArbitrageBucket"

# Folder containing JSON files
folder_path = "/Users/priyasuresh/Documents/Documents_Priya’s MacBook Pro/SPRING2024/Data 228-Big Data/project/last_folder/clusters.json"
old_json_path = "/Users/priyasuresh/Documents/Documents_Priya’s MacBook Pro/SPRING2024/Data 228-Big Data/project/old_json"

# Ensure the old_json folder exists
os.makedirs(old_json_path, exist_ok=True)

# Initialize the InfluxDB client
client = InfluxDBClient(url=influxdb_url, token=influxdb_token, org=influxdb_org)
write_api = client.write_api()  # If SYNCHRONOUS is available, add it inside write_api()

def load_json_files_to_influxdb():
    # Get a list of all JSON files in the directory
    for filename in os.listdir(folder_path):
        if filename.endswith(".json") and not filename.startswith("_"):
            full_path = os.path.join(folder_path, filename)
            try:
                with open(full_path, 'r') as file:
                    for line in file:
                        try:
                            record = json.loads(line)
                            # Create a new Point object for InfluxDB
                            point = (
                                Point("crypto_trade")
                                .tag("symbol_id", record.get("symbol_id", ""))
                                .tag("taker_side", record.get("taker_side", "UNKNOWN"))
                                .field("price", float(record.get("price", 0)))
                                .field("min_buy_price", float(record.get("min_buy_price", 0)))
                                .field("max_sell_price", float(record.get("max_sell_price", 0)))
                                .field("price_diff", float(record.get("price_diff", 0)))
                                .time(record.get("time_exchange", 0), WritePrecision.MS)
                            )
                            # Write the point to InfluxDB
                            write_api.write(bucket=influxdb_bucket, record=point)
                        except json.JSONDecodeError as json_err:
                            print(f"JSON error in file {filename}: {json_err}")
                        except Exception as e:
                            print(f"An error occurred processing record in file {filename}: {e}")
                print(f"Data from {filename} has been written to InfluxDB.")
                shutil.move(full_path, os.path.join(old_json_path, filename))
            except Exception as file_err:
                print(f"An error occurred with file {filename}: {file_err}")

# Continuously check the folder every 5 minutes
try:
    while True:
        load_json_files_to_influxdb()
        # Sleep for 5 minutes
        time.sleep(300)
except KeyboardInterrupt:
    print("Data loading interrupted by user.")
finally:
    client.close()
    print("InfluxDB client closed.")

# Remember to replace 'your_influxdb_token' with your actual InfluxDB token.


Data from part-00004-f47e6e0c-4c7c-4d79-b9d1-33a5cfda8b85-c000.json has been written to InfluxDB.
Data from part-00001-f47e6e0c-4c7c-4d79-b9d1-33a5cfda8b85-c000.json has been written to InfluxDB.
Data from part-00003-f47e6e0c-4c7c-4d79-b9d1-33a5cfda8b85-c000.json has been written to InfluxDB.
Data from part-00006-f47e6e0c-4c7c-4d79-b9d1-33a5cfda8b85-c000.json has been written to InfluxDB.
Data from part-00007-f47e6e0c-4c7c-4d79-b9d1-33a5cfda8b85-c000.json has been written to InfluxDB.
Data from part-00002-f47e6e0c-4c7c-4d79-b9d1-33a5cfda8b85-c000.json has been written to InfluxDB.
Data from part-00000-f47e6e0c-4c7c-4d79-b9d1-33a5cfda8b85-c000.json has been written to InfluxDB.
Data from part-00005-f47e6e0c-4c7c-4d79-b9d1-33a5cfda8b85-c000.json has been written to InfluxDB.
Data from part-00008-f47e6e0c-4c7c-4d79-b9d1-33a5cfda8b85-c000.json has been written to InfluxDB.
Data from part-00003-e7fe12a4-ecbb-40e7-8b67-2822b8c8cbf8-c000.json has been written to InfluxDB.
Data from part-00006

In [1]:
pip install influxdb-client


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 744.6/744.6 kB 8.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.8/217.8 kB 29.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
